# Submission of Calculation with Subsequent Results Query

In this demo, we will submit a calculation, check the status of the job and query the results after it is finished.

## Initialization of Workspace 

Specify the Azure quantum workspace and the 'connection string' which allows us to connect to the workspace

In [ ]:
from azure.quantum import Workspace
from azure.quantum.job import JobFailedWithResultsError

# insert connection string from Azure Portal Workspace Access Keys
connection_string = "" 
workspace = Workspace.from_connection_string(connection_string)

In [ ]:
# To submit Accelerated DFT jobs, we will be using the microsoft.dft target in the workspace.
print("Verifying access to Accelerated DFT target.")
target = workspace.get_targets("microsoft.dft")
print("Verification complete.")

## Define Input and Submit Accelerated DFT job

In [ ]:
# First, let's define the molecular structure, loaded from an xyz file.
from pathlib import Path
GeomFile = "molecules/aspirin.xyz"

In [ ]:
# Secondly, let's give a name for the job.
job_name = 'aspirin_spe'

In [ ]:
# Next, we create a dictionary variable to specify the parameters for the DFT calculation. 
dft_input_params = {
  "tasks": [
    {
      "taskType": "spe", 
      "basisSet": { "name": 'def2-svpd'},
      "xcFunctional": { "name": "b3lyp", "gridLevel": 4 },
      "molecule": { "charge": 0, "multiplicity": 1 },
      "scf": { "method": "rks", "maxSteps": 100, "convergeThreshold": 1e-8, "requireWaveFunction": True }
      # example with PCM solvent and D3 dispersion correction
      #"scf":{"method":"rks","dispersion":"d3zero","convergeThreshold":1e-8,"pcm":{"solverType":"iefpcm","solvent":"water"}}
    }
  ]
}

# We are now ready to submit the Job using the target.submit call. It takes three parameters-
# 1. The input molecule in xyz format.
# 2. The DFT parameters that we declared above.
# 3. A friendly name to help identify the job in the Azure Portal later.

print("Submitting DFT job.")

job = target.submit(
    input_data=Path(GeomFile).read_text(),
    input_params = dft_input_params,
    name= job_name)
    
print("\nDFT job has been submitted.")
print(f"\nJob name: {job_name}")


Show the status of the job. If the job has finished, read the results of the job

In [ ]:
job.refresh()
print(f'Job: "{job_name}" is {job.details.status}')
if job.details.status == 'Succeeded':
    qcschema = job.get_results()["results"][0]

## Results

The results of the calculation are stored in the QCSchema format dict.

For an SPE calculation we can see the energy by simply looking at the key "return_result".
(For SPF calculations, this key returns the force)

In [ ]:
print("SPE Result: ",qcschema["return_result"])

Other useful information is stored in the output dict, for example:

In [ ]:
print("Number of Basis Functions: ", qcschema["properties"]["calcinfo_nbasis"])
print("Total Energy (Hartree): ", qcschema["properties"]["return_energy"])
print("Nuclear Repulsion Energy (Hartree): ", qcschema["properties"]["nuclear_repulsion_energy"])
print("Total Calculation Time (s): ", qcschema["provenance"]["total_time_seconds"])

The output can be explored using qcschema.keys()

Wavefunction information is also saved in the output if "requireWaveFunction": True was set.
The "wavefunction" key contains orbitals, orbital energies, orbital occupancies, and Fock matrices.
This will be used in our later examples for property calculations.

### Output Results to QCSchema json file

Saving to a json-formatted file makes it easy to read/write/visualize the QCSchema key structure.  

In [ ]:
import json
qcschema_json = job_name + "_output.json"
with open(qcschema_json, "w") as fp:
    json.dump(qcschema, fp)

# Visualization and Property Calculation

The key "requireWaveFunction": True in our input (above) instructs Accelerated DFT to store the wavefunction information.
This is information can be easily read by PySCF an allows us to use PySCF property and visualization tools.

In [ ]:
import pyscf
from tools.libqcschema import *
from pyscf import gto, dft
import json
import numpy as np

# Create PySCF DFT object
mol, ks = recreate_scf_obj(qcschema)

# note: PySCF prints 'ECP def2-svpd not found for  C' etc, this is expected.

We have now loaded the Accelerated DFT result into a PySCF object, we can now use PySCF tools to generate cube files. The cube files are used to view molecular orbitals and electron density (via py3Dmol).

In [ ]:
# pyscf cube generation tool:
from pyscf.tools import cubegen

# For visualization of molecules and orbitals:
import py3Dmol
from tools.visualize import niceview

## Molecular Orbitals

Here we are visualizing the HOMO, which is the 47th orbital (index 46 when starting counting from 0)

In [ ]:
# index of the orbital of interest (starting from 0):
mo_index = 46

mo_file = 'mo.cube'
cubegen.orbital(mol, mo_file, ks.mo_coeff[:,mo_index])

data = None
with open(mo_file, "r") as infile:
    data = infile.read()
    
view = py3Dmol.view()
niceview(view,data)
view.addModel(data, "cube")
view.setStyle({"stick": {}})
view.zoomTo()
view.show()

### Aside: How to tell which orbitals are the HOMO/LUMO?
There are a few options:

1. look at the occupations of alpha and beta electrons in qcschema["wavefunction"]["scf_occupations_a"] and qcschema["wavefunction"]["scf_occupations_b"].
2. Look at the number of alpha and beta electrons using qcschema["properties"]["calcinfo_nalpha"] and qcschema["properties"]["calcinfo_nbeta"] and the spin
3. Use Mulliken analysis and look at the populations

In [ ]:
# For this closed shell system (number of alpha electrons = number of beta electrons):
for i in range(len(qcschema["wavefunction"]["scf_occupations_a"])):
    if(qcschema["wavefunction"]["scf_occupations_a"][i] < 1.0):
        LUMO = i
        HOMO = i-1
        break

gap = qcschema["wavefunction"]["scf_eigenvalues_a"][HOMO] - qcschema["wavefunction"]["scf_eigenvalues_a"][LUMO]
print("HOMO index: ",HOMO)
print("LUMO index: ",LUMO)
print(f"HOMO-LUMO gap: {gap:.6f}"," Hartree")

# Property Calculation: RESP Charges

We can compute partitioned atomic charges in several ways, including Mulliken charges, CHELPG charges and RESP charges.
In this example we use RESP charges.

## RESP Charges

In [ ]:
import tools.resp
from tools.resp import resp

q_resp = resp(ks)
print("charges: ", q_resp)


We can see the assigned charges more easily with a diagram from rdkit

In [ ]:
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from rdkit.Chem import rdDetermineBonds
IPythonConsole.drawOptions.addAtomIndices = False
IPythonConsole.molSize = 300,300

raw_mol = Chem.MolFromXYZFile('molecules/aspirin.xyz')
mol = Chem.Mol(raw_mol)
rdDetermineBonds.DetermineConnectivity(mol)

# For each atom, get the charge computed with CHELPG/RESP
for atom in mol.GetAtoms():
    index = atom.GetIdx()
    pchrg = q_resp[index]
    atom.SetProp("atomNote", str(round(pchrg, 2))) #str(pchrg))
    
mol